In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
#tf.compat.v1.enable_eager_execution()
tf.enable_eager_execution()
import keras.preprocessing.image as process_im
from keras.applications import vgg19
from keras.models import Model
from tensorflow.python.keras import models
from tensorflow.python.keras import losses
from tensorflow.python.keras import layers
from tensorflow.python.keras import backend as k
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import functools
import IPython.display
import PIL

In [ ]:
import zipfile

In [ ]:
zipfilename="vgg19_model.zip"
print(zipfile.is_zipfile(zipfilename))

In [ ]:
zfile = zipfile.ZipFile(zipfilename, 'r')
print(zfile)

In [ ]:
with zipfile.ZipFile(zipfilename,'r')as file:
    print(zfile.infolist())
outpath='./'

In [ ]:
with zipfile.ZipFile(zipfilename, 'r') as zfile:

    nl = zfile.namelist()
    il = zfile.infolist()

    for id,l in enumerate(nl):
        zfile.extract(l, path=outpath)
        print(l)

In [ ]:
import tensorflow as tf
model = tf.keras.applications.vgg19.VGG19(include_top=False,weights=None)
model.load_weights('./vgg19_model/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')

In [ ]:
content_path='./work/YellowLabradorLooking_new.jpg'
style_path='./work/Vassily_Kandinsky,_1913_-_Composition_7.jpg'

In [ ]:
def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim

    new_shape = tf.cast(shape * scale, tf.int32)

    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

In [ ]:
def imshow(image, title=None):
    if len(image.shape) > 3:
        image = tf.squeeze(image, axis=0)

    plt.imshow(image)
    if title:
        plt.title(title)

In [ ]:
content=load_img(content_path)
plt.subplot(1, 2, 1)
imshow(content, 'Content Image')
style=load_img(style_path)
plt.subplot(1, 2, 1)
imshow(style, 'Style Image')
style.shape

In [ ]:
x = tf.keras.applications.vgg19.preprocess_input(content*255)
x = tf.image.resize(x, (448, 448))
prediction_probabilities = model(x)
prediction_probabilities.shape

In [ ]:
print()
for layer in model.layers:
    print(layer.name)

In [ ]:
content_layers = ['block5_conv2']
style_layers = ['block1_conv1',
                'block2_conv1',
                'block3_conv1',
                'block4_conv1',
                'block5_conv1']
num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

In [ ]:
def vgg_layers(layer_names):
    model = tf.keras.applications.vgg19.VGG19(include_top=False,weights=None)
    model.load_weights('./vgg19_model/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5')
    model.trainable = False

    outputs = [model.get_layer(name).output for name in layer_names]

    model_ = tf.keras.Model([model.input], outputs)
    return model_

In [ ]:
style_extractor = vgg_layers(style_layers)
style_outputs = style_extractor(style*255)

#Look at the statistics of each layer's output
for name, output in zip(style_layers, style_outputs):
    print(name)
    print("  shape: ", output.numpy().shape)
    print("  min: ", output.numpy().min())
    print("  max: ", output.numpy().max())
    print("  mean: ", output.numpy().mean())
    print()

In [ ]:
def gram_matrix(input_tensor):
    result = tf.linalg.einsum('bijc,bijd->bcd', input_tensor, input_tensor)
    input_shape = tf.shape(input_tensor)
    num_locations = tf.cast(input_shape[1]*input_shape[2], tf.float32)
    return result/(num_locations)

In [ ]:
class StyleContentModel(tf.keras.models.Model):
    def __init__(self, style_layers, content_layers):
        super(StyleContentModel, self).__init__()
        self.vgg = vgg_layers(style_layers + content_layers)
        self.style_layers = style_layers
        self.content_layers = content_layers
        self.num_style_layers = len(style_layers)
        self.vgg.trainable = False

    def call(self, inputs):
        inputs = inputs*255.0
        preprocessed_input = tf.keras.applications.vgg19.preprocess_input(inputs)
        outputs = self.vgg(preprocessed_input)
        style_outputs, content_outputs = (outputs[:self.num_style_layers],
                                          outputs[self.num_style_layers:])

        style_outputs = [gram_matrix(style_output)
                         for style_output in style_outputs]

        content_dict = {content_name: value
                        for content_name, value
                        in zip(self.content_layers, content_outputs)}

        style_dict = {style_name: value
                      for style_name, value
                      in zip(self.style_layers, style_outputs)}

        return {'content': content_dict, 'style': style_dict}

In [ ]:
extractor = StyleContentModel(style_layers, content_layers)

results = extractor(tf.constant(content))

print('Styles:')
for name, output in sorted(results['style'].items()):
    print("  ", name)
    print("    shape: ", output.numpy().shape)
    print("    min: ", output.numpy().min())
    print("    max: ", output.numpy().max())
    print("    mean: ", output.numpy().mean())
    print()

print("Contents:")
for name, output in sorted(results['content'].items()):
    print("  ", name)
    print("    shape: ", output.numpy().shape)
    print("    min: ", output.numpy().min())
    print("    max: ", output.numpy().max())
    print("    mean: ", output.numpy().mean())

In [ ]:
content_img=tf.Variable(content)
style_img=tf.Variable(style)

In [ ]:
def tensor_to_image(tensor):
    tensor = tensor*255
    tensor = np.array(tensor, dtype=np.uint8)
    if np.ndim(tensor)>3:
        assert tensor.shape[0] == 1
        tensor = tensor[0]
    return PIL.Image.fromarray(tensor)

In [ ]:
tensor_to_image(content_img)

In [ ]:
tensor_to_image(style_img)

In [ ]:
def clip_0_1(image):
    return tf.clip_by_value(image, clip_value_min=0.0, clip_value_max=1.0)

In [ ]:
def style_content_loss(outputs):
    style_outputs = outputs['style']
    content_outputs = outputs['content']
    style_loss = tf.add_n([tf.reduce_mean((style_outputs[name]-style_targets[name])**2) 
                           for name in style_outputs.keys()])
    style_loss *= style_weight / num_style_layers
    content_loss = tf.add_n([tf.reduce_mean((content_outputs[name]-content_targets[name])**2) 
                             for name in content_outputs.keys()])
    content_loss *= content_weight / num_content_layers
    loss = style_loss + content_loss
    return loss

In [ ]:
style_targets = extractor(style)['style']
content_targets = extractor(content)['content']

In [ ]:
image = tf.Variable(content)

In [ ]:
tensor_to_image(image)

In [ ]:
opt = tf.keras.optimizers.Adam(learning_rate=0.02, beta_1=0.99, epsilon=1e-1)

In [ ]:
style_weight=1e-2
content_weight=1e4

In [ ]:
@tf.function()
def train_step(image):
    with tf.GradientTape() as tape:
        outputs = extractor(image)
        loss = style_content_loss(outputs)

    grad = tape.gradient(loss, image)
    opt.apply_gradients([(grad, image)])
    image.assign(clip_0_1(image))

In [ ]:
epoches=20

In [ ]:
def rms_loss(image1,image2):
    loss = tf.reduce_mean(input_tensor=tf.square(image1 - image2))
    return loss

In [ ]:
def content_loss(content, target):
    return rms_loss(content, target)

In [ ]:
def style_loss(style, gram_target):
    gram_style = gram_matrix(style)
    return rms_loss(gram_style, gram_target)

In [ ]:
train_step(image)
print('第一次训练后的loss函数为：',float(content_loss(content,image))
      ,float(style_loss(style,style_weight)))
train_step(image)
print('第二次训练后的loss函数为：',float(content_loss(content,image))
      ,float(style_loss(style,style_weight)))
train_step(image)
print('第三次训练后的loss函数为：',float(content_loss(content,image))
      ,float(style_loss(style,style_weight)))
train_step(image)
print('第四次训练后的loss函数为：',float(content_loss(content,image))
      ,float(style_loss(style,style_weight)))
train_step(image)
loss=style_content_loss(extractor(image))
print('第五次训练后的loss函数为',float(content_loss(content,image))
      ,float(style_loss(style,style_weight)))
tensor_to_image(image)

In [ ]:
for i in range(epoches):
    train_step(image)
    print('第',i+6,'次训练的loss函数为：',float(content_loss(content,image))
          ,float(style_loss(image,style_weight)))
    

In [ ]:
import time
start = time.time()
steps_per_epoch = 95
step = 0
for m in range(steps_per_epoch):
    step += 1
    train_step(image)
    print(".", end='', flush=True)
end = time.time()
print("Total time: {:.1f}".format(end-start))

In [ ]:
loss_C=content_loss(content,image)
loss_S=style_loss(style,style_weight)

In [ ]:
print(float(loss_C))

In [ ]:
print(float(loss_S))